In [2]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get OPENAI_API_KEY and OPENAI_ORGANIZATION from .env file
api_key = os.getenv("OPENAI_API_KEY")
organization = os.getenv("OPENAI_ORGANIZATION")
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"), organization=os.getenv("OPENAI_ORGANIZATION")
)

In [3]:
messages = [
    {
        "role": "system",
        "content": "You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
    },
    {"role": "user", "content": "Hello!"},
]

In [4]:
from typing import List, Dict

class Message:
    def __init__(
        self,
        role: str,
        content: str,
        tool_calls: List=[],
        tool_call_id: str = None,
        name: str = None
    ):
        self.role = role
        self.content = content
        self.tool_calls = tool_calls
        self.tool_call_id = tool_call_id
        self.name = name
        
        
    def to_dict(self):
        message_dict = {'role': self.role, 'content': self.content}
        if self.tool_calls:
            message_dict['tool_calls'] = self.tool_calls
        if self.tool_call_id:
            message_dict['tool_call_id'] = self.tool_call_id
        if self.name:
            message_dict['name'] = self.name
        return message_dict
    
    
class Conversation:
    def __init__(self, messages: List[Dict[str,str]]):
        self.messages = [Message(**message) for message in messages]
        
    def __iter__(self):
        for message in self.messages:
            yield message
            
    def to_dict(self):
        return [message.to_dict() for message in self.messages]
    
    def add_message(self, message: Dict[str,str]):
        self.messages.append(Message(**message))
        
    def add_system_message(self, completion):
        self.add_message(
            {
                'role' : completion.choices[0].message.role,
                'content' : completion.choices[0].message.content,
                'tool_calls' : completion.choices[0].message.tool_calls,
            }
        )
        
    def add_human_message(self, message):
        self.add_message(
            {
                'role' : 'user',
                'content' : message
            }
        )
        
    def add_tool_message(self, tool_called, response):
        self.add_message(
            {
                'role' : 'tool', 
                'name' : tool_called.function.name,
                'tool_call_id': tool_called.id,
                'content' : str(response)
            }
        )
        
    def print_conversation(self): 
        for message in self.messages:
            print(f"{message.role} : {message.content}")
            
            

In [5]:
conversation = Conversation(messages)
conversation.print_conversation()

system : You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
user : Hello!


In [6]:
conversation.to_dict()

[{'role': 'system',
  'content': "You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user', 'content': 'Hello!'}]

In [7]:
def chat_with_openai(conversation):
    completion = client.chat.completions.create(
        model = 'gpt-3.5-turbo-1106',
        messages = conversation.to_dict()
    )
    return completion


In [8]:
completion = chat_with_openai(conversation=conversation)
print(completion.choices[0].message)
print(completion)

ChatCompletionMessage(content='Hi there! How can I assist you today with adding and multiplying numbers together?', role='assistant', function_call=None, tool_calls=None)
ChatCompletion(id='chatcmpl-91x6vQSXe0lTo4EjZIZCsLsglRFbd', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hi there! How can I assist you today with adding and multiplying numbers together?', role='assistant', function_call=None, tool_calls=None))], created=1710252577, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_f93e21ed76', usage=CompletionUsage(completion_tokens=16, prompt_tokens=46, total_tokens=62))


Here it goes we add with proceeding Manually


In [9]:
conversation.add_system_message(completion=completion)

conversation.print_conversation()

system : You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
user : Hello!
assistant : Hi there! How can I assist you today with adding and multiplying numbers together?


In [10]:
conversation.add_human_message('I want to get to know physics')
conversation.print_conversation()

system : You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
user : Hello!
assistant : Hi there! How can I assist you today with adding and multiplying numbers together?
user : I want to get to know physics


In [11]:
completion = chat_with_openai(conversation)
conversation.add_system_message(completion)
conversation.print_conversation()

system : You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
user : Hello!
assistant : Hi there! How can I assist you today with adding and multiplying numbers together?
user : I want to get to know physics
assistant : That's great! Physics is a fascinating subject. Are there specific concepts or topics within physics that you're interested in learning about? For example, are you interested in classical mechanics, electromagnetism, thermodynamics, quantum mechanics, or any other specific area of physics?


Process it indefinetely


In [12]:
def run_conversation(new_message, conversation):
    conversation.add_human_message(new_message)
    completion = chat_with_openai(conversation)
    conversation.add_system_message(completion)
    conversation.print_conversation()
    return conversation

run_conversation("go for it", conversation)

system : You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
user : Hello!
assistant : Hi there! How can I assist you today with adding and multiplying numbers together?
user : I want to get to know physics
assistant : That's great! Physics is a fascinating subject. Are there specific concepts or topics within physics that you're interested in learning about? For example, are you interested in classical mechanics, electromagnetism, thermodynamics, quantum mechanics, or any other specific area of physics?
user : go for it
assistant : Alright! Let's start with a classic physics equation. The equation for force (F) is given by F = m * a, where m is the mass of an object and a is its acceleration. If you have the values for mass and acceleration, I can help you calculate the force acting on the object using this equation.


# ADD Tools

In [13]:
def add_two_numbers(args):
    return args['number_a'] + args['number_b']

Specify the Tools


In [29]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "add_two_numbers",
            "description": "Add two numbers together",
            "parameters": {
                "type": "object",
                "properties": {
                    "number_a": {
                        "type": "integer",
                        "description": "The first number",
                    },
                    "number_b": {
                        "type": "integer",
                        "description": "The second number.",
                    },
                },
                "required": ["number_a", "number_b"],
            },
        }
    }
]

In [30]:
def chat_with_openai_using_tools(conversation):
    completion = client.chat.completions.create(
        model = 'gpt-3.5-turbo-1106',
        messages = conversation.to_dict(),
        tools = tools
    )
    return completion

In [31]:
# We want our messages to look something like this:
messages = [
    {
        "role": "system",
        "content": "You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. Use your tools when they are helpful",
    },
    {"role": "user", "content": "Help me add 13 and 27"},
]

conversation_with_tools = Conversation(messages)


In [32]:
conversation_with_tools.to_dict()

[{'role': 'system',
  'content': "You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. Use your tools when they are helpful"},
 {'role': 'user', 'content': 'Help me add 13 and 27'}]

In [33]:
#  call it now with our convesation
completion = chat_with_openai_using_tools(conversation_with_tools)
conversation_with_tools.add_system_message(completion)
print(completion.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_GimrKeXBNoZpgsLHDPBNk2jy', function=Function(arguments='{"number_a":13,"number_b":27}', name='add_two_numbers'), type='function')])


In [34]:
tool_called = completion.choices[0].message.tool_calls[0]
print(tool_called.function.name)
print(tool_called.function.arguments)
print(tool_called.id)

add_two_numbers
{"number_a":13,"number_b":27}
call_GimrKeXBNoZpgsLHDPBNk2jy


In [35]:
conversation_with_tools.to_dict()

[{'role': 'system',
  'content': "You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. Use your tools when they are helpful"},
 {'role': 'user', 'content': 'Help me add 13 and 27'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [ChatCompletionMessageToolCall(id='call_GimrKeXBNoZpgsLHDPBNk2jy', function=Function(arguments='{"number_a":13,"number_b":27}', name='add_two_numbers'), type='function')]}]

In [36]:
# Let's call our amazing addition function
import json

response = add_two_numbers(json.loads(tool_called.function.arguments))
print(response)

40


In [37]:
conversation_with_tools.add_tool_message(tool_called, response)
conversation_with_tools.to_dict()

[{'role': 'system',
  'content': "You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. Use your tools when they are helpful"},
 {'role': 'user', 'content': 'Help me add 13 and 27'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [ChatCompletionMessageToolCall(id='call_GimrKeXBNoZpgsLHDPBNk2jy', function=Function(arguments='{"number_a":13,"number_b":27}', name='add_two_numbers'), type='function')]},
 {'role': 'tool',
  'content': '40',
  'tool_call_id': 'call_GimrKeXBNoZpgsLHDPBNk2jy',
  'name': 'add_two_numbers'}]

In [38]:
# Let's call it now with our convesation
completion = chat_with_openai_using_tools(conversation_with_tools)

#conversation_with_tools.print_conversation()

In [39]:
conversation_with_tools.add_system_message(completion)

In [40]:
conversation_with_tools.print_conversation()

system : You are an expert at adding and multiplying numbers together. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. Use your tools when they are helpful
user : Help me add 13 and 27
assistant : None
tool : 40
assistant : The sum of 13 and 27 is 40.


# USE LANGCHAIN AGENTS